In [1]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.neighbors import NearestNeighbors
from sklearn.decomposition import PCA

import requests
from bs4 import BeautifulSoup
import pandas as pd
import itertools
import re
from tqdm import tqdm_notebook
import string
import numpy as np

df_final = pd.read_csv('coursera.csv').drop('Unnamed: 0',axis=1)
df_stack = pd.read_csv('tidy_questions.csv').drop('Unnamed: 0', axis=1).set_index('Id').dropna()

df_final['level'] = df_final['level'].fillna('unknown')

df_final['Text'] = df_final["about"] + df_final["syllabus"]
df_final['index'] = df_final['index'] + ' ' + df_final['level'] + ' ' + df_final['rating'].astype('str') + ' ' + df_final['rating_count'].astype('str')
df_train = df_final[["index", "Text"]]

df_train.set_index('index', drop = True, inplace=True)

df_cnn = pd.concat([df_train, df_stack], axis=0)

#Define the TFIDF vectorizer that will be used to process the data
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df_cnn['Text'])
tfidf_courses = tfidf_vectorizer.transform(df_train['Text'])
tfidf_questions = tfidf_vectorizer.transform(df_stack['Text'])

nbrs1 = NearestNeighbors(n_neighbors=533).fit(tfidf_matrix)
nbrs_courses1 = NearestNeighbors(n_neighbors=533).fit(tfidf_courses)
nbrs_questions1 = NearestNeighbors(n_neighbors=533).fit(tfidf_questions)

def get_closest_neighs1(name, target='course'):
    if target == 'course':
        nbrs_local1 = nbrs_courses1
    elif target == 'question':
        nbrs_local1 = nbrs_questions1
    elif target == 'multi':
        nbrs_local1 = nbrs1
    row = df_cnn.index.get_loc(name)
    distances, indices = nbrs_local1.kneighbors(tfidf_matrix.getrow(row))
    indexes_similar = pd.Series(indices.flatten()).map(df_cnn.reset_index()['index'])
    result = pd.DataFrame({'distance': distances.flatten(), 'index': indexes_similar})
    
    return result

#pred1 = get_closest_neighs(id_q, target='course')

In [2]:
import pandas as pd
df_qu = pd.read_csv('coursera_ques.csv')
posts = pd.read_csv("Posts.csv",  encoding = "ISO-8859-1").dropna(subset=['Tags'])[['Id', 'Tags']].values

df_qu.tail()

from sklearn.neighbors import NearestNeighbors

idx = 533

courses = df_qu.values[:idx,:]
questions = df_qu.values[idx:, :]

questions[:,0]

tags_df = pd.read_csv('Tags.csv',  encoding = "ISO-8859-1" )
tags = tags_df.sort_values('Count', ascending=False)['TagName'].values[:500]

questions.shape

tags.shape

tags_indexes = dict([(tag, index) for index, tag in enumerate(tags)])

import numpy as np
from scipy.sparse import csr_matrix

array = np.zeros((courses.shape[0],tags.shape[0]*2) , dtype=np.int8)
for i in range(courses.shape[0]):
    for j in range(tags.shape[0]):
        if tags[j] in courses[i, 1]:
            array[i, tags_indexes[tags[j]]] = 1
            array[i, tags_indexes[tags[j]] + tags.shape[0]] = 1

courses_df = pd.DataFrame(array)
courses_df['id'] = courses[:,0]
courses_df.set_index('id',inplace=True)

array1 = np.zeros((questions.shape[0],tags.shape[0]*2) , dtype=np.int8)
for i in range(questions.shape[0]):
    for j in range(tags.shape[0]):
        if tags[j] in questions[i, 1]:
            array1[i, tags_indexes[tags[j]]] = 1
        if tags[j] in posts[i, 1]:
            array1[i, tags_indexes[tags[j]] + tags.shape[0]] = 1

questions_df = pd.DataFrame(array1)
questions_df['id'] = questions[:,0]
questions_df.set_index('id',inplace=True)

union = csr_matrix(pd.concat([courses_df, questions_df],axis=0))
questions_df = csr_matrix(questions_df)
courses_df = csr_matrix(courses_df)

tags = None
tags_indexes = None
array = None
array1 = None

import gc
gc.collect()

nbrs2 = NearestNeighbors(n_neighbors=533).fit(union)
nbrs_courses2 = NearestNeighbors(n_neighbors=533).fit(courses_df)
nbrs_questions2 = NearestNeighbors(n_neighbors=533).fit(questions_df)

df_qu = df_qu.set_index('Unnamed: 0')

def get_closest_neighs2(name, target='course'):
    if target == 'course':
        nbrs_local2 = nbrs_courses2
    elif target == 'question':
        nbrs_local2 = nbrs_questions2
    elif target == 'multi':
        nbrs_local2 = nbrs2
    row = df_qu.index.get_loc(name)
    distances, indices = nbrs_local2.kneighbors(union.getrow(row))
    indexes_similar = pd.Series(indices.flatten()).map(df_qu.reset_index()['Unnamed: 0'])
    result = pd.DataFrame({'distance': distances.flatten(), 'index': indexes_similar})
    
    return result

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (3,18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [25]:
from sklearn.preprocessing import StandardScaler

def get_real_lol(id):
    tmpget2 = get_closest_neighs2(str(id))
    tmpget1 = get_closest_neighs1(int(id))
    sscaler = StandardScaler()

    data1 = tmpget1.set_index('index')
    data2 = tmpget2.set_index('index')
    data1['distance'] = sscaler.fit_transform(data1['distance'].values)
    data2['distance'] = sscaler.fit_transform(data2['distance'].values)
    data1['sum'] = data1['distance'] + data2['distance']
    data1 = data1.sort_values('sum')
    datatmp = data1.index[:3]
    res = ""
    for i in datatmp:
        res += i.split(" ")[0] + '\n'
    return res

getting random


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:586: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:649: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:586: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single fea

id got


2017-12-03 13:34:56,580 - telegram.ext.dispatcher - ERROR - An uncaught error was raised while processing the update
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\pandas\indexes\base.py", line 2134, in get_loc
    return self._engine.get_loc(key)
  File "pandas\index.pyx", line 132, in pandas.index.IndexEngine.get_loc (pandas\index.c:4433)
  File "pandas\index.pyx", line 154, in pandas.index.IndexEngine.get_loc (pandas\index.c:4279)
  File "pandas\src\hashtable_class_helper.pxi", line 732, in pandas.hashtable.PyObjectHashTable.get_item (pandas\hashtable.c:13742)
  File "pandas\src\hashtable_class_helper.pxi", line 740, in pandas.hashtable.PyObjectHashTable.get_item (pandas\hashtable.c:13696)
KeyError: 'https:'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\telegram\ext\dispatcher.py", line 270, in process_update
    handler.handle_update(u

In [81]:
debug_tmp = ""

import random
import pandas as pd
def get_qid(solink):
    return solink.split("/")[4]
pref = "data/NLP_dataset_final/"

courses_links_csv = pref + "courses_links" + debug_tmp + ".csv"

courses_links_csv = 'coursera_ques.csv'



def c_link_by_qid(solink):
    link = ""
    link = get_real_lol(solink)
    if (link == ""):
        return "Not found"
    return link.strip()

def get_random_id():
    num_lines = sum(1 for line in open(courses_links_csv))
    r = random.randrange(600, num_lines)
    for i, line in enumerate(open(courses_links_csv)):
        if i == 0:
            continue
        if i == r:
            lnk = line.split(" ")[0].split(',')[0]
            return lnk
    return "Can not calculate"
    

from telegram.ext import Updater
updater = Updater(token='473606748:AAEQecsSQ3Xk6Cwl-yBgTcV1juIV0Fj5mCY')
def start(bot, update):
    res = "examples\n"
    with open(courses_links_csv) as courses_links_df:
        count = 0
        for line in courses_links_df.readlines():
            count += 1
            if (count < 600):
                continue
            if count > 605:
                break
            print(line)
            qid = line.split(",")[0]
            res += "https://stackoverflow.com/questions/" + qid + '\n'
    bot.send_message(chat_id=update.message.chat_id, text=res)
dispatcher = updater.dispatcher
import logging
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
                level=logging.INFO)
from telegram.ext import CommandHandler
start_handler = CommandHandler('start', start)
dispatcher.add_handler(start_handler)

def echo(bot, update):
    
    qid = get_qid(update.message.text)
    bot.send_message(chat_id=update.message.chat_id, text=c_link_by_qid(qid))
    
def random_links(bot, update):
    print('getting random')
    rid = get_random_id()
    print(rid)
    print('id got')
    print('random got')
    stov = "https://stackoverflow.com/questions/" + rid + '\n'
    bot.send_message(chat_id=update.message.chat_id, text=c_link_by_qid(rid))
    bot.send_message(chat_id=update.message.chat_id, text=stov)
    
start_handler = CommandHandler('random', random_links)
dispatcher.add_handler(start_handler)

from telegram.ext import MessageHandler, Filters
echo_handler = MessageHandler(Filters.text, echo)
dispatcher.add_handler(echo_handler)

In [ ]:
updater.start_polling()

In [80]:
updater.stop()